Webscraping mit Pyhon: https://realpython.com/beautiful-soup-web-scraper-python/#scrape-the-fake-python-job-site

In [1]:
import requests
import re
from bs4 import BeautifulSoup
from databaseHelper import Person

persons = []

UHH

In [2]:
uhh_url = "https://www2.informatik.uni-hamburg.de/fiona/pers.php"

page = requests.get(uhh_url)
soup = BeautifulSoup(page.content, "html.parser")

result = []
# get all <tr> elements (here: rows of the person table)
for person in soup.find_all(name="tr"):
    # only <tr> elements describing actual persons have an id attribute
    if not person.has_attr("id"):
        continue

    # all table field values are insde <a> tags
    fields = person.findAll("a")

    name = fields[1].text
    department = fields[2].text

    result.append(Person(
        name=name,
        university="Universität Hamburg",
        department=department
    ))

print(f"done: scraped {len(result)} UHH staff")
persons.extend(result)

done: scraped 325 UHH staff


HAW

In [3]:
haw_url = "https://www.haw-hamburg.de/hochschule/technik-und-informatik/departments/informatik/unser-department/beschaeftigte/"

page = requests.get(haw_url)
soup = BeautifulSoup(page.content, "html.parser")

result = []

for row in soup.findAll("div", class_="row person-tile filter_tile border-top pt-4"):
    person = row.find("div", class_="col-12")

    name = ""

    for tag in person.findAll("a", {"title" : "Zur Profilseite"}):
        for part in tag.findAll("b"):
            name += " " + re.sub(r"\s+", " ", part.text.strip())
        result.append(Person(
            name=name.strip(),
            university="HAW Hamburg"
        ))

print(f"done: scraped {len(result)} HAW staff")
persons.extend(result)

done: scraped 67 HAW staff


split name into title and name

for instance: 
`"Prof. Dr. Klaus-Peter Kossakowski"` becomes `"Prof. Dr.", "Klaus-Peter Kossakowski"`

In [4]:
for person in persons:
    # chatGPT kann super so dumme regexe schreiben 
    pattern = r"^(Prof\. Dr\.|Prof\.|Dr\.)(-Ing\.| rer\. nat\.)?(?=.)(?:\s|$)"
    
    title = re.search(pattern, person.name)
    
    if not title: continue

    person.title = title.group().strip()
    person.name = re.sub(pattern, "", person.name).strip()

insert persons into database

In [5]:
from databaseHelper import PersonDatabaseHelper

dbHelper = PersonDatabaseHelper()

dbHelper.deleteTable()
dbHelper.createTable()

dbHelper.insertPersons(persons)

dbHelper.commit()

inserted 392 rows into table person


In [ ]:
for person in dbHelper.selectAll():
    print(person)